In [1]:
#%pip install torch==2.5.1 torchvision==0.20.1 torchaudio==2.5.1 --index-url https://download.pytorch.org/whl/cu124 --force-reinstall

In [2]:
import torch

def format_pytorch_version(version_str):
    # Example input: "2.0.1+cu118" -> returns "2.0.1"
    return version_str.split('+')[0]

def format_cuda_version(cuda_str):
    # If CUDA is None (CPU-only PyTorch), return "cpu"
    if cuda_str is None:
        return "cpu"
    # Example: "11.8" -> "cu118"
    return "cu" + cuda_str.replace('.', '')

TORCH_version = torch.__version__
TORCH = format_pytorch_version(TORCH_version)
CUDA_version = torch.version.cuda
CUDA = format_cuda_version(CUDA_version)

print(f"Installing packages for torch-{TORCH}+{CUDA}...")

%pip install --upgrade --no-cache-dir pyg_lib torch_scatter torch_sparse torch_cluster torch_spline_conv -f https://data.pyg.org/whl/torch-{TORCH}+{CUDA}.html

%pip install torch_geometric

Installing packages for torch-2.5.1+cu124...
Looking in links: https://data.pyg.org/whl/torch-2.5.1+cu124.html
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install scikit-learn imbalanced-learn xgboost

Note: you may need to restart the kernel to use updated packages.


In [4]:
%pip install optuna

Note: you may need to restart the kernel to use updated packages.


### Hyperparameter Optimization Citeseer
#### SCAR 

In [1]:
from train_NNIF_GNN import run_nnif_gnn_experiment
import optuna
from typing import Dict, Any

def objective(trial: optuna.trial.Trial) -> float:
    """
    Objective function for hyperparameter optimization using Optuna.
    It builds a parameter dictionary, calls the experiment function, and
    returns the average F1 score.
    """
    params: Dict[str, Any] = {
        "dataset_name": "citeseer",      
        "mechanism": "SCAR",
        "train_pct": 0.5,
        "K": trial.suggest_int("K", 2, 7),
        "layers": trial.suggest_int("layers", 1, 2),
        "hidden_channels": trial.suggest_categorical("hidden_channels", [64,128, 256]),
        "out_channels": trial.suggest_categorical("out_channels", [64,128, 256]),
        "norm": None,
        "dropout": 0,
        "ratio": trial.suggest_float("ratio", 0.09, 0.18),
        "aggregation": trial.suggest_categorical("aggregation", ['sum', 'mean']),
        "treatment": trial.suggest_categorical("treatment", ["removal", "relabeling"]),
        "model_type": trial.suggest_categorical("model_type", ['GCNConv', 'GATConv', 'SAGEConv', 'GINConv']),
        "rate_pairs":trial.suggest_int("rate_pairs",1,15),
        "batch_size": trial.suggest_categorical("batch_size", [1,5,10,15]),
        "lr": trial.suggest_float("lr", 1e-4, 1e-2),
        "reliable_mini_batch":True,
        "seeds": 5,
        "output_csv": "citeseer_scar_sampling_nnif_batch_cluster.csv",
        "min":0.835,
        "clusters":trial.suggest_categorical("clusters", [100,200,300,400,500]),
       
    }
    
    avg_f1, std_f1 = run_nnif_gnn_experiment(params)
    
    # We aim to maximize F1 score.
    return avg_f1

# Create an Optuna study to maximize the F1 score.
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=200)

# Print out the best hyperparameters and corresponding F1 score.
print("Best trial:")
trial = study.best_trial
print("  Average F1:", trial.value)
print("  Best parameters:")
for key, value in trial.params.items():
    print(f"    {key}: {value}")


[I 2025-03-12 11:20:21,411] A new study created in memory with name: no-name-c7765fdb-3828-49f5-bbd8-fe3279a0a332


Running experiment with seed=654:
 - K=4, layers=2, hidden=256, out=64
 - norm=None, dropout=0, batch_size=15, reliable_mini_batch=True
 - ratio=0.09767575025383361, aggregation=sum, treatment=relabeling
 - model_type=SAGEConv, rate_pairs=10, clusters=100, lr=0.00901690252087701


Computing METIS partitioning...
Done!


Epoch 0, Loss: 4.0619
Epoch 10, Loss: 3.1184
Epoch 20, Loss: 2.0205
Epoch 30, Loss: 1.3516
Epoch 40, Loss: 0.9404


[I 2025-03-12 11:20:52,031] Trial 0 finished with value: 0.676602086438152 and parameters: {'K': 4, 'layers': 2, 'hidden_channels': 256, 'out_channels': 64, 'ratio': 0.09767575025383361, 'aggregation': 'sum', 'treatment': 'relabeling', 'model_type': 'SAGEConv', 'rate_pairs': 10, 'batch_size': 15, 'lr': 0.00901690252087701, 'clusters': 100}. Best is trial 0 with value: 0.676602086438152.


...

Epoch 0, Loss: 17.1991
Epoch 10, Loss: 14.7375
Epoch 20, Loss: 13.6470
Epoch 30, Loss: 12.9946
Epoch 40, Loss: 10.6316
 - Metrics: Accuracy=0.9456, F1=0.8691, Recall=0.8573, Precision=0.8812
Running experiment with seed=281:
 - K=5, layers=1, hidden=128, out=256
 - norm=None, dropout=0, batch_size=5, reliable_mini_batch=True
 - ratio=0.15537689941662486, aggregation=sum, treatment=removal
 - model_type=GATConv, rate_pairs=2, clusters=100, lr=0.001238920580803676


Computing METIS partitioning...
Done!


Epoch 0, Loss: 17.1929
Epoch 10, Loss: 14.7337
Epoch 20, Loss: 13.6323
Epoch 30, Loss: 12.9963
Epoch 40, Loss: 10.6272


[I 2025-03-12 18:35:19,276] Trial 199 finished with value: 0.8631811744781113 and parameters: {'K': 5, 'layers': 1, 'hidden_channels': 128, 'out_channels': 256, 'ratio': 0.15537689941662486, 'aggregation': 'sum', 'treatment': 'removal', 'model_type': 'GATConv', 'rate_pairs': 2, 'batch_size': 5, 'lr': 0.001238920580803676, 'clusters': 100}. Best is trial 182 with value: 0.8670499182943534.


 - Metrics: Accuracy=0.9414, F1=0.8571, Recall=0.8345, Precision=0.8810
Done. Results written to citeseer_experimentations\citeseer_scar_sampling_nnif_batch_cluster_1203183111.csv.
Average F1 over valid seeds: 0.8632 ± 0.0043
Best trial:
  Average F1: 0.8670499182943534
  Best parameters:
    K: 5
    layers: 1
    hidden_channels: 128
    out_channels: 256
    ratio: 0.15537792511092066
    aggregation: sum
    treatment: removal
    model_type: GATConv
    rate_pairs: 1
    batch_size: 10
    lr: 0.0012727616268830802
    clusters: 100


### Hyperparameter Optimization Citeseer
#### SAR 

In [ ]:
from train_NNIF_GNN import run_nnif_gnn_experiment
import optuna
from typing import Dict, Any

def objective(trial: optuna.trial.Trial) -> float:
    """
    Objective function for hyperparameter optimization using Optuna.
    It builds a parameter dictionary, calls the experiment function, and
    returns the average F1 score.
    """
    params: Dict[str, Any] = {
        "dataset_name": "citeseer",      
        "mechanism": "SAR",
        "train_pct": 0.5,
        "K": trial.suggest_int("K", 2, 7),
        "layers": trial.suggest_int("layers", 1, 2),
        "hidden_channels": trial.suggest_categorical("hidden_channels", [64,128, 256]),
        "out_channels": trial.suggest_categorical("out_channels", [64,128, 256]),
        "norm": None,
        "dropout": 0,
        "ratio": trial.suggest_float("ratio", 0.09, 0.18),
        "aggregation": trial.suggest_categorical("aggregation", ['sum', 'mean']),
        "treatment": trial.suggest_categorical("treatment", ["removal", "relabeling"]),
        "model_type": trial.suggest_categorical("model_type", ['GCNConv', 'GATConv', 'SAGEConv', 'GINConv']),
        "rate_pairs":trial.suggest_int("rate_pairs",1,10),
        "batch_size": trial.suggest_categorical("batch_size", [128,256,512,1024,2048]),
        "lr": trial.suggest_float("lr", 1e-4, 1e-2),
        "reliable_mini_batch":True,
        "seeds": 5,
        "output_csv": "citeseer_sar_sampling_nnif_batch_cluster.csv",
        "min":0.82
    }
    
    avg_f1, std_f1 = run_nnif_gnn_experiment(params)
    
    # We aim to maximize F1 score.
    return avg_f1

# Create an Optuna study to maximize the F1 score.
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=200)

# Print out the best hyperparameters and corresponding F1 score.
print("Best trial:")
trial = study.best_trial
print("  Average F1:", trial.value)
print("  Best parameters:")
for key, value in trial.params.items():
    print(f"    {key}: {value}")


### Hyperparameter Optimization Cora
#### SCAR 

In [2]:
from train_NNIF_GNN import run_nnif_gnn_experiment
import optuna
from typing import Dict, Any

def objective(trial: optuna.trial.Trial) -> float:
    """
    Objective function for hyperparameter optimization using Optuna.
    It builds a parameter dictionary, calls the experiment function, and
    returns the average F1 score.
    """
    params: Dict[str, Any] = {
        "dataset_name": "cora",      
        "mechanism": "SCAR",
        "train_pct": 0.5,
        "K": trial.suggest_int("K", 4, 12),
        "layers": trial.suggest_int("layers", 1, 2),
        "hidden_channels": trial.suggest_categorical("hidden_channels", [64,128, 256]),
        "out_channels": trial.suggest_categorical("out_channels", [64,128, 256]),
        "norm": None,
        "dropout": 0,
        "ratio": trial.suggest_float("ratio", 0.16, 0.34),
        "aggregation": trial.suggest_categorical("aggregation", ['sum', 'mean']),
        "treatment": trial.suggest_categorical("treatment", ["removal", "relabeling"]),
        "model_type": trial.suggest_categorical("model_type", ['GCNConv', 'GATConv', 'SAGEConv', 'GINConv']),
        "rate_pairs":trial.suggest_int("rate_pairs",1,15),
        "batch_size": trial.suggest_categorical("batch_size", [1,5,10,15]),
        "lr": trial.suggest_float("lr", 1e-4, 1e-2),
        "reliable_mini_batch":True,
        "seeds": 5,
        "output_csv": "cora_scar_sampling_nnif_batch_cluster.csv",
        "min":0.84,
        "clusters":trial.suggest_categorical("clusters", [100,200,300,400,500]),
       
    }
    
    avg_f1, std_f1 = run_nnif_gnn_experiment(params)
    
    # We aim to maximize F1 score.
    return avg_f1

# Create an Optuna study to maximize the F1 score.
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=200)

# Print out the best hyperparameters and corresponding F1 score.
print("Best trial:")
trial = study.best_trial
print("  Average F1:", trial.value)
print("  Best parameters:")
for key, value in trial.params.items():
    print(f"    {key}: {value}")


[I 2025-03-12 18:35:19,310] A new study created in memory with name: no-name-c6e6107b-69a9-4980-b4f1-03e43f44daf4


Running experiment with seed=654:
 - K=10, layers=1, hidden=256, out=64
 - norm=None, dropout=0, batch_size=10, reliable_mini_batch=True
 - ratio=0.26316520760366796, aggregation=mean, treatment=removal
 - model_type=GINConv, rate_pairs=13, clusters=500, lr=0.005158738530547523


Computing METIS partitioning...
Done!


Epoch 0, Loss: 102.2856
Epoch 10, Loss: 32.8988
Epoch 20, Loss: 11.9643
Epoch 30, Loss: 5.5308
Epoch 40, Loss: 3.2635


[I 2025-03-12 18:36:59,417] Trial 0 finished with value: 0.7504553734061931 and parameters: {'K': 10, 'layers': 1, 'hidden_channels': 256, 'out_channels': 64, 'ratio': 0.26316520760366796, 'aggregation': 'mean', 'treatment': 'removal', 'model_type': 'GINConv', 'rate_pairs': 13, 'batch_size': 10, 'lr': 0.005158738530547523, 'clusters': 500}. Best is trial 0 with value: 0.7504553734061931.


...

 - Metrics: Accuracy=0.9357, F1=0.8945, Recall=0.9022, Precision=0.8870
Done. Results written to cora_experimentations\cora_scar_sampling_nnif_batch_cluster_1303015741.csv.
Average F1 over valid seeds: 0.8921 ± 0.0024
Running experiment with seed=654:
 - K=9, layers=2, hidden=64, out=128
 - norm=None, dropout=0, batch_size=15, reliable_mini_batch=True
 - ratio=0.21982739566283738, aggregation=mean, treatment=relabeling
 - model_type=GCNConv, rate_pairs=5, clusters=100, lr=0.00566850839924623


Computing METIS partitioning...
Done!


Epoch 0, Loss: 8.4219
Epoch 10, Loss: 6.5729
Epoch 20, Loss: 5.6080
Epoch 30, Loss: 4.5227
Epoch 40, Loss: 3.9312


[I 2025-03-13 02:02:04,817] Trial 199 finished with value: 0.7626728110599078 and parameters: {'K': 9, 'layers': 2, 'hidden_channels': 64, 'out_channels': 128, 'ratio': 0.21982739566283738, 'aggregation': 'mean', 'treatment': 'relabeling', 'model_type': 'GCNConv', 'rate_pairs': 5, 'batch_size': 15, 'lr': 0.00566850839924623, 'clusters': 100}. Best is trial 162 with value: 0.9152454984972689.


 - Metrics: Accuracy=0.8479, F1=0.7627, Recall=0.8093, Precision=0.7211
F1 = 0.76 < 0.84, skipping ...
Done. Results written to cora_experimentations\cora_scar_sampling_nnif_batch_cluster_1303020142.csv.
Average F1 over valid seeds: 0.7627 ± 0.0000
Best trial:
  Average F1: 0.9152454984972689
  Best parameters:
    K: 9
    layers: 2
    hidden_channels: 128
    out_channels: 128
    ratio: 0.21953508533558438
    aggregation: mean
    treatment: removal
    model_type: GCNConv
    rate_pairs: 6
    batch_size: 15
    lr: 0.004752812661789618
    clusters: 100


### Hyperparameter Optimization Cora
#### SAR 

In [ ]:
from train_NNIF_GNN import run_nnif_gnn_experiment
import optuna
from typing import Dict, Any

def objective(trial: optuna.trial.Trial) -> float:
    """
    Objective function for hyperparameter optimization using Optuna.
    It builds a parameter dictionary, calls the experiment function, and
    returns the average F1 score.
    """
    params: Dict[str, Any] = {
        "dataset_name": "cora",      
        "mechanism": "SAR",
        "train_pct": 0.5,
        "K": trial.suggest_int("K", 4, 12),
        "layers": trial.suggest_int("layers", 1, 2),
        "hidden_channels": trial.suggest_categorical("hidden_channels", [64,128, 256]),
        "out_channels": trial.suggest_categorical("out_channels", [64,128, 256]),
        "norm": None,
        "dropout": 0,
        "ratio": trial.suggest_float("ratio", 0.16, 0.34),
        "aggregation": trial.suggest_categorical("aggregation", ['sum', 'mean']),
        "treatment": trial.suggest_categorical("treatment", ["removal", "relabeling"]),
        "model_type": trial.suggest_categorical("model_type", ['GCNConv', 'GATConv', 'SAGEConv', 'GINConv']),
        "rate_pairs":trial.suggest_int("rate_pairs",1,10),
        "batch_size": trial.suggest_categorical("batch_size", [128,256,512,1024,2048]),
        "lr": trial.suggest_float("lr", 1e-4, 1e-2),
        "reliable_mini_batch":True,
        "seeds": 5,
        "output_csv": "cora_sar_sampling_nnif_batch_cluster.csv",
        "min":0.84
    }
    
    avg_f1, std_f1 = run_nnif_gnn_experiment(params)
    
    # We aim to maximize F1 score.
    return avg_f1

# Create an Optuna study to maximize the F1 score.
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=200)

# Print out the best hyperparameters and corresponding F1 score.
print("Best trial:")
trial = study.best_trial
print("  Average F1:", trial.value)
print("  Best parameters:")
for key, value in trial.params.items():
    print(f"    {key}: {value}")


### Hyperparameter Optimization Pubmed
#### SCAR 

In [3]:
from train_NNIF_GNN import run_nnif_gnn_experiment
import optuna
from typing import Dict, Any

def objective(trial: optuna.trial.Trial) -> float:
    """
    Objective function for hyperparameter optimization using Optuna.
    It builds a parameter dictionary, calls the experiment function, and
    returns the average F1 score.
    """
    params: Dict[str, Any] = {
        "dataset_name": "pubmed",      
        "mechanism": "SCAR",
        "train_pct": 0.5,
        "K": trial.suggest_int("K", 5, 14),
        "layers": trial.suggest_int("layers", 1, 2),
        "hidden_channels": trial.suggest_categorical("hidden_channels", [64,128, 256]),
        "out_channels": trial.suggest_categorical("out_channels", [64,128, 256]),
        "norm": None,
        "dropout": 0,
        "ratio": trial.suggest_float("ratio", 0.2, 0.4),
        "aggregation": trial.suggest_categorical("aggregation", ['sum', 'mean']),
        "treatment": trial.suggest_categorical("treatment", ["removal", "relabeling"]),
        "model_type": trial.suggest_categorical("model_type", ['GCNConv', 'GATConv', 'SAGEConv', 'GINConv']),
        "rate_pairs":trial.suggest_int("rate_pairs",1,15),
        "batch_size": trial.suggest_categorical("batch_size", [1,5,10,15]),
        "lr": trial.suggest_float("lr", 1e-4, 1e-2),
        "reliable_mini_batch":True,
        "seeds": 5,
        "output_csv": "pubmed_scar_sampling_nnif_batch_cluster.csv",
        "min":0.84,
        "clusters":trial.suggest_categorical("clusters", [100,200,300,400,500]),
       
    }
    
    avg_f1, std_f1 = run_nnif_gnn_experiment(params)
    
    # We aim to maximize F1 score.
    return avg_f1

# Create an Optuna study to maximize the F1 score.
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=200)

# Print out the best hyperparameters and corresponding F1 score.
print("Best trial:")
trial = study.best_trial
print("  Average F1:", trial.value)
print("  Best parameters:")
for key, value in trial.params.items():
    print(f"    {key}: {value}")


[I 2025-03-13 02:02:04,849] A new study created in memory with name: no-name-571831ee-fc80-404f-ace4-7a2a2b732141


Running experiment with seed=654:
 - K=12, layers=1, hidden=128, out=64
 - norm=None, dropout=0, batch_size=5, reliable_mini_batch=True
 - ratio=0.34140521086160325, aggregation=sum, treatment=relabeling
 - model_type=GATConv, rate_pairs=11, clusters=200, lr=0.006258973153089475


Computing METIS partitioning...
Done!


Epoch 0, Loss: 30.7130
Epoch 10, Loss: 8.2699
Epoch 20, Loss: 2.5205
Epoch 30, Loss: 1.2462
Epoch 40, Loss: 0.7852


[I 2025-03-13 02:03:48,826] Trial 0 finished with value: 0.771375029280862 and parameters: {'K': 12, 'layers': 1, 'hidden_channels': 128, 'out_channels': 64, 'ratio': 0.34140521086160325, 'aggregation': 'sum', 'treatment': 'relabeling', 'model_type': 'GATConv', 'rate_pairs': 11, 'batch_size': 5, 'lr': 0.006258973153089475, 'clusters': 200}. Best is trial 0 with value: 0.771375029280862.


...

Epoch 0, Loss: 13.3482
Epoch 10, Loss: 11.3809
Epoch 20, Loss: 6.2114
Epoch 30, Loss: 3.1893
Epoch 40, Loss: 1.9449
 - Metrics: Accuracy=0.8914, F1=0.8663, Recall=0.8806, Precision=0.8524
Running experiment with seed=281:
 - K=14, layers=2, hidden=256, out=256
 - norm=None, dropout=0, batch_size=10, reliable_mini_batch=True
 - ratio=0.3337441603777864, aggregation=sum, treatment=removal
 - model_type=GCNConv, rate_pairs=2, clusters=100, lr=0.00787790063643142


Computing METIS partitioning...
Done!


Epoch 0, Loss: 13.3482
Epoch 10, Loss: 11.3809
Epoch 20, Loss: 6.2114
Epoch 30, Loss: 3.1893
Epoch 40, Loss: 1.9449


[I 2025-03-14 02:04:35,409] Trial 199 finished with value: 0.866752052960116 and parameters: {'K': 14, 'layers': 2, 'hidden_channels': 256, 'out_channels': 256, 'ratio': 0.3337441603777864, 'aggregation': 'sum', 'treatment': 'removal', 'model_type': 'GCNConv', 'rate_pairs': 2, 'batch_size': 10, 'lr': 0.00787790063643142, 'clusters': 100}. Best is trial 113 with value: 0.8799741426716741.


 - Metrics: Accuracy=0.8934, F1=0.8685, Recall=0.8817, Precision=0.8558
Done. Results written to pubmed_experimentations\pubmed_scar_sampling_nnif_batch_cluster_1403015959.csv.
Average F1 over valid seeds: 0.8668 ± 0.0015
Best trial:
  Average F1: 0.8799741426716741
  Best parameters:
    K: 12
    layers: 2
    hidden_channels: 256
    out_channels: 256
    ratio: 0.24401391061267885
    aggregation: mean
    treatment: removal
    model_type: GATConv
    rate_pairs: 3
    batch_size: 5
    lr: 0.00869255050992657
    clusters: 100


### Hyperparameter Optimization Pubmed
#### SAR 

In [ ]:
from train_NNIF_GNN import run_nnif_gnn_experiment
import optuna
from typing import Dict, Any

def objective(trial: optuna.trial.Trial) -> float:
    """
    Objective function for hyperparameter optimization using Optuna.
    It builds a parameter dictionary, calls the experiment function, and
    returns the average F1 score.
    """
    params: Dict[str, Any] = {
        "dataset_name": "pubmed",      
        "mechanism": "SAR",
        "train_pct": 0.5,
        "K": trial.suggest_int("K", 6, 14),
        "layers": trial.suggest_int("layers", 1, 2),
        "hidden_channels": trial.suggest_categorical("hidden_channels", [64, 128, 256]),
        "out_channels": trial.suggest_categorical("out_channels", [64, 128, 256]),
        "norm": None,
        "dropout": 0,
        "ratio": trial.suggest_float("ratio", 0.23, 0.27),
        "aggregation": trial.suggest_categorical("aggregation", ['sum', 'mean']),
        "treatment": trial.suggest_categorical("treatment", ["removal", "relabeling"]),
        "model_type": trial.suggest_categorical("model_type", ['GCNConv', 'GATConv', 'SAGEConv', 'GINConv']),
        "rate_pairs":trial.suggest_int("rate_pairs",1,10),
        "batch_size": trial.suggest_categorical("batch_size", [512,1024,2048,5096,10192]),
        "lr": trial.suggest_float("lr", 1e-5, 1e-2),
        "reliable_mini_batch":True,
        "seeds": 5,
        "output_csv": "pubmed_sar_sampling_nnif_batch_cluster.csv",
        "min": 0.82
        }
    
    # Call the experiment function with these parameters.
    # run_nnif_gnn_experiment returns (avg_f1, std_f1)
    avg_f1, std_f1 = run_nnif_gnn_experiment(params)
    
    # We aim to maximize F1 score.
    return avg_f1

# Create an Optuna study to maximize the F1 score.
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=200)

# Print out the best hyperparameters and corresponding F1 score.
print("Best trial:")
trial = study.best_trial
print("  Average F1:", trial.value)
print("  Best parameters:")
for key, value in trial.params.items():
    print(f"    {key}: {value}")


### Hyperparameter Optimization WikiCS
#### SCAR 

In [ ]:
from train_NNIF_GNN import run_nnif_gnn_experiment
import optuna
from typing import Dict, Any

def objective(trial: optuna.trial.Trial) -> float:
    """
    Objective function for hyperparameter optimization using Optuna.
    It builds a parameter dictionary, calls the experiment function, and
    returns the average F1 score.
    """
    params: Dict[str, Any] = {
        "dataset_name": "wiki-cs",      
        "mechanism": "SCAR",
        "train_pct": 0.5,
        "K": trial.suggest_int("K", 3, 40),
        "layers": trial.suggest_int("layers", 1, 2),
        "hidden_channels": trial.suggest_categorical("hidden_channels", [64,128, 256]),
        "out_channels": trial.suggest_categorical("out_channels", [64,128, 256]),
        "norm": None,
        "dropout": 0,
        "ratio": trial.suggest_float("ratio", 0.09, 0.18),
        "aggregation": trial.suggest_categorical("aggregation", ['sum', 'mean']),
        "treatment": trial.suggest_categorical("treatment", ["removal", "relabeling"]),
        "model_type": trial.suggest_categorical("model_type", ['GCNConv', 'GATConv', 'SAGEConv', 'GINConv']),
        "rate_pairs":trial.suggest_int("rate_pairs",1,10),
        "batch_size": trial.suggest_categorical("batch_size", [1,5,10,15]),
        "lr": trial.suggest_float("lr", 1e-4, 1e-2),
        "reliable_mini_batch":True,
        "seeds": 5,
        "output_csv": "wikics_scar_sampling_nnif_batch_cluster.csv",
        "min":0.88,
        "clusters":trial.suggest_categorical("clusters", [100,200,300,400,500]),
       
    }
    
    avg_f1, std_f1 = run_nnif_gnn_experiment(params)
    
    # We aim to maximize F1 score.
    return avg_f1

# Create an Optuna study to maximize the F1 score.
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=200)

# Print out the best hyperparameters and corresponding F1 score.
print("Best trial:")
trial = study.best_trial
print("  Average F1:", trial.value)
print("  Best parameters:")
for key, value in trial.params.items():
    print(f"    {key}: {value}")


[I 2025-03-14 02:04:35,449] A new study created in memory with name: no-name-e5dcf471-fcac-4aab-8f6d-b3b093666d23


Running experiment with seed=654:
 - K=8, layers=2, hidden=128, out=128
 - norm=None, dropout=0, batch_size=5, reliable_mini_batch=True
 - ratio=0.11008449308317064, aggregation=mean, treatment=removal
 - model_type=GCNConv, rate_pairs=3, clusters=100, lr=0.001451234283197598


Computing METIS partitioning...
Done!


Epoch 0, Loss: 33.5343
Epoch 10, Loss: 29.7977
Epoch 20, Loss: 27.8999
Epoch 30, Loss: 23.6593
Epoch 40, Loss: 19.9143
 - Metrics: Accuracy=0.9574, F1=0.9014, Recall=0.8499, Precision=0.9595
Running experiment with seed=114:
 - K=8, layers=2, hidden=128, out=128
 - norm=None, dropout=0, batch_size=5, reliable_mini_batch=True
 - ratio=0.11008449308317064, aggregation=mean, treatment=removal
 - model_type=GCNConv, rate_pairs=3, clusters=100, lr=0.001451234283197598


Computing METIS partitioning...
Done!


...

Computing METIS partitioning...
Done!


Epoch 0, Loss: 20.9089
Epoch 10, Loss: 19.5268
Epoch 20, Loss: 16.6540
Epoch 30, Loss: 15.6314
Epoch 40, Loss: 14.6815


[I 2025-03-14 14:57:53,054] Trial 199 finished with value: 0.9197740892805021 and parameters: {'K': 35, 'layers': 2, 'hidden_channels': 256, 'out_channels': 256, 'ratio': 0.16810415247537985, 'aggregation': 'mean', 'treatment': 'removal', 'model_type': 'GCNConv', 'rate_pairs': 7, 'batch_size': 15, 'lr': 0.0025304429458340204, 'clusters': 100}. Best is trial 163 with value: 0.9244167623455898.


 - Metrics: Accuracy=0.9634, F1=0.9204, Recall=0.9242, Precision=0.9167
Done. Results written to wiki-cs_experimentations\wikics_scar_sampling_nnif_batch_cluster_1403145358.csv.
Average F1 over valid seeds: 0.9198 ± 0.0010
Best trial:
  Average F1: 0.9244167623455898
  Best parameters:
    K: 34
    layers: 2
    hidden_channels: 256
    out_channels: 256
    ratio: 0.17331567524348823
    aggregation: mean
    treatment: removal
    model_type: GCNConv
    rate_pairs: 8
    batch_size: 15
    lr: 0.0018990508573961478
    clusters: 100


: 

### Hyperparameter Optimization WikiCS
#### SAR 

In [ ]:
from train_NNIF_GNN import run_nnif_gnn_experiment
import optuna
from typing import Dict, Any

def objective(trial: optuna.trial.Trial) -> float:
    """
    Objective function for hyperparameter optimization using Optuna.
    It builds a parameter dictionary, calls the experiment function, and
    returns the average F1 score.
    """
    params: Dict[str, Any] = {
        "dataset_name": "wiki-cs",      
        "mechanism": "SAR",
        "train_pct": 0.5,
        "alpha": 0.5,
        "K": trial.suggest_int("K", 23, 48),
        "layers": trial.suggest_int("layers", 1, 2),
        "hidden_channels": trial.suggest_categorical("hidden_channels", [64,128, 256]),
        "out_channels": trial.suggest_categorical("out_channels", [64,128, 256]),
        "norm": None,
        "dropout": 0,#trial.suggest_float("dropout", 0, 0.5),
        "margin": 0.5,
        "lpl_weight": 0.5,#trial.suggest_float("lpl_weight", 0.01, 0.99),
        "ratio": trial.suggest_float("ratio", 0.08, 0.48),
        "aggregation": trial.suggest_categorical("aggregation", ['sum', 'mean']),
        "treatment": trial.suggest_categorical("treatment", ["removal", "relabeling"]),
        "model_type": trial.suggest_categorical("model_type", ['GCNConv', 'GATConv', 'SAGEConv', 'GINConv', 'TransformerConv']),
        "sampling_mode":trial.suggest_categorical("sampling_mode", [None,'nn', 'random', 'feature']), #'weighted',
        "num_neighbors": trial.suggest_int("num_neighbors", 5, 20),
        "seeds": 5,
        "output_csv": "wikics_sar_conv_sampling.csv"
    }
    
    # Call the experiment function with these parameters.
    # run_nnif_gnn_experiment returns (avg_f1, std_f1)
    avg_f1, std_f1 = run_nnif_gnn_experiment(params)
    
    # We aim to maximize F1 score.
    return avg_f1

# Create an Optuna study to maximize the F1 score.
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=200)

# Print out the best hyperparameters and corresponding F1 score.
print("Best trial:")
trial = study.best_trial
print("  Average F1:", trial.value)
print("  Best parameters:")
for key, value in trial.params.items():
    print(f"    {key}: {value}")


# Elliptic Bitcoin

In [ ]:
from train_NNIF_GNN import run_nnif_gnn_experiment
import optuna
from typing import Dict, Any

def objective(trial: optuna.trial.Trial) -> float:
    """
    Objective function for hyperparameter optimization using Optuna.
    It builds a parameter dictionary, calls the experiment function, and
    returns the average F1 score.
    """
    params: Dict[str, Any] = {
        "dataset_name": "elliptic-bitcoin",      
        "mechanism": "SAR2",
        "train_pct": 0.5,
        "alpha": 0.5,
        "K": trial.suggest_int("K", 3, 20),
        "layers": trial.suggest_int("layers", 1, 3),
        "hidden_channels": trial.suggest_categorical("hidden_channels", [64,128, 256]),
        "out_channels": trial.suggest_categorical("out_channels", [64,128, 256]),
        "norm": None,
        "dropout": trial.suggest_float("dropout", 0, 0.4),
        "ratio": trial.suggest_float("ratio", 0.001, 0.03),
        "aggregation": trial.suggest_categorical("aggregation", ['sum', 'mean']),
        "treatment": trial.suggest_categorical("treatment", ["removal", "relabeling"]),
        "model_type": trial.suggest_categorical("model_type", ['GCNConv', 'GATConv', 'SAGEConv', 'GINConv']),
        "rate_pairs":trial.suggest_int("rate_pairs",1,10),
        "batch_size": trial.suggest_categorical("batch_size", [10,20,30,40]),
        "lr": trial.suggest_float("lr", 1e-4, 1e-2),
        "reliable_mini_batch":True,
        "seeds": 1,
        "output_csv": "elliptic_bitcoin_cluster.csv",
        "min":0.65,
        "clusters":trial.suggest_categorical("clusters", [100,500,1000,1500])
        }

    
    # Call the experiment function with these parameters.
    # run_nnif_gnn_experiment returns (avg_f1, std_f1)
    avg_f1, std_f1 = run_nnif_gnn_experiment(params)
    
    # We aim to maximize F1 score.
    return avg_f1

# Create an Optuna study to maximize the F1 score.
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=200)

# Print out the best hyperparameters and corresponding F1 score.
print("Best trial:")
trial = study.best_trial
print("  Average F1:", trial.value)
print("  Best parameters:")
for key, value in trial.params.items():
    print(f"    {key}: {value}")


[I 2025-03-17 16:34:38,505] A new study created in memory with name: no-name-04a81f50-1ac7-4b23-887b-16249a07773e


Running experiment with seed=654:
 - K=6, layers=1, hidden=64, out=256
 - norm=None, dropout=0.08550861157228283, batch_size=30, reliable_mini_batch=True
 - ratio=0.0028386665790275396, aggregation=sum, treatment=relabeling
 - model_type=GINConv, rate_pairs=8, clusters=500, lr=0.004842931069794964


Computing METIS partitioning...
